In [2]:
import requests
import geopandas as gpd
import json
from progress.bar import Bar
from tqdm import trange, tqdm
from enum import IntEnum
import pandas as pd

In [8]:
# TODO extract to arcgis helper functions, then check with multiple URLs


# url = "https://geowb.worldbank.org/server/rest/services/GLOBAL/Global_Admin_Units/MapServer/2/query?where=&geometry=geometry%3D%7Bxmin%3A+-110%2C+ymin%3A+20%2C+xmax%3A+-90.32%2C+ymax%3A+41%7D&outFields=*&returnGeometry=true&f=json"
# extra = "&geometryType=esriGeometryEnvelope&spatialRel=esriSpatialRelIntersects"

BASE_URL = "https://geowb.worldbank.org/server/rest/services/GLOBAL/Global_Admin_Units/MapServer"

class AdminLevel(IntEnum):
    COUNTRY = 2
    ADMIN_1 = 1
    ADMIN_2 = 0

# BBox is of format [xmin, ymin, xmax, ymax]

def get_boundaries_by_bbox(bbox: list[int], admin_level: int, ids_only=False, where="", debug=False, as_gdf=False):
    assert admin_level in [0,1,2]
    assert len(bbox) == 4

    url = f"{BASE_URL}/{admin_level}/query?"

    res = requests.get(url, params={
        "geometry": ",".join([str(xy) for xy in bbox]),
        "outFields": "*",
        "f": "json",
        "where": where,
        "returnIdsOnly": str(ids_only).lower()
    })

    if debug:
        # TODO more here
        print(url)

    return gpd.read_file(res.text) if as_gdf else res.json()

def get_ids_by_bbox(bbox: list[int], admin_level: int, where=''):
    res_dict = get_boundaries_by_bbox(bbox, admin_level, ids_only=True, where=where)
    return res_dict["objectIds"]

def get_empty_structure(admin_level):
    structure = get_boundaries_by_bbox([0, 0, 1, 1], admin_level)
    structure["features"] = []
    return structure

def get_boundary_by_id(id: int, admin_level: int):
    assert admin_level in [0,1,2]
    url = f"{BASE_URL}/{admin_level}/{id}?f=json"
    res = requests.get(url)
    return res.json()

def get_boundaries_overlapping_bbox(bbox: list[int], admin_level: int, debug=False, as_gdf=False):
    ids = get_ids_by_bbox(bbox, admin_level)
    structure = get_empty_structure(admin_level)

    if debug:
        ids = tqdm(ids)
    
    for _, id in enumerate(ids):
        res = get_boundary_by_id(id, admin_level)
        if res.get('error') and debug:
            print(res)
            break

        structure["features"].append(res["feature"])
    
    return gpd.read_file(json.dumps(structure)) if as_gdf else structure

In [9]:
countries_gdf = get_boundaries_overlapping_bbox([0,-0,20,20], AdminLevel.COUNTRY, debug=True, as_gdf=True)
countries_gdf

100%|██████████| 16/16 [00:10<00:00,  1.56it/s]


,OBJECTID_1,WB_ADM0_NA,ISO3,ISO_A2,IFC_region,geometry
0,3,Algeria,DZA,DZ,EMENA,"MULTIPOLYGON (((-1.12515 35.73673, -1.13231 35..."
1,25,Benin,BEN,BJ,CAF,"POLYGON ((2.83915 12.40646, 2.84301 12.40350, ..."
2,37,Burkina Faso,BFA,BF,CAF,"POLYGON ((-0.45567 15.08082, -0.44206 15.06811..."
3,40,Cameroon,CMR,CM,CAF,"MULTIPOLYGON (((9.63469 3.60798, 9.63279 3.606..."
4,44,Central African Republic,CAF,CF,CAF,"POLYGON ((22.87551 10.93137, 22.87363 10.92280..."
5,45,Chad,TCD,TD,CAF,"POLYGON ((24.00000 19.50000, 24.00000 16.49706..."
6,53,Congo,COG,CG,CAF,"POLYGON ((17.60632 3.64125, 17.65328 3.63904, ..."
7,61,Democratic Republic of Congo,COD,CD,CAF,"MULTIPOLYGON (((12.43123 -6.01024, 12.43191 -6..."
8,69,Equatorial Guinea,GNQ,GQ,CAF,"MULTIPOLYGON (((5.62957 -1.45728, 5.62430 -1.4..."
9,79,Gabon,GAB,GA,CAF,"MULTIPOLYGON (((8.83154 -0.92271, 8.83809 -0.9..."
